<a href="https://colab.research.google.com/github/Guhan-12/FakeNewsPredictionProject/blob/main/FakeNewsPredictionML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:

id: unique id for a news article

title: the title of a news article

author: author of the news article

text: the text of the article; could be incomplete

label: a label that marks whether the news article is real or fake:

        1 - Fake news
        0 - Real news


Importing Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re  # regular expresion - to search for a text in a doc
from nltk.corpus import stopwords  # nltk - natural language took kit
from nltk.stem.porter import PorterStemmer # Stemming - removes prefix and suffix from a word
from sklearn.feature_extraction.text import TfidfVectorizer # convert data into feature vectors
from sklearn.model_selection import train_test_split  # split dataset to train data and test data
from sklearn.linear_model import LogisticRegression # to train the model
from sklearn.metrics import accuracy_score # to evaluate the accuracy of the train data and test data
from sklearn.preprocessing import LabelEncoder

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# printing the stopwords in the english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [ ]:
# loading the dataset into the dataframe
def bad_line(x):
    return None

news_dataset = pd.read_csv('/content/train.csv', encoding='utf-8', on_bad_lines=bad_line, engine='python')
#news_dataset = pd.read_csv('/content/train.csv',encoding = 'utf-8')

In [ ]:
# to display the number of data in the dataset
news_dataset.shape


(6875, 5)

In [ ]:
# to display the first 5 data of the dataset
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# to count the number values in each column
news_dataset.isnull().sum()


id          0
title     193
author    659
text       14
label       0
dtype: int64

In [ ]:
# replacing null values with empty string
news_dataset = news_dataset.fillna('')

In [ ]:
# to create a new column by merging two existing columns
news_dataset['contents'] = news_dataset['author'] + ' ' + news_dataset['title']

In [ ]:
# to display the newly created column
print(news_dataset['contents'])

0       Darrell Lucus House Dem Aide: We Didn’t Even S...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
6870    Cara Buckley Andrew Garfield, Spiritual Ghost ...
6871    Jameson Parker Harry Reid Vindicated: The FBI ...
6872    Jack Hadfield Over 12,000 Tweets Call For Trum...
6873    Ray McGovern Putin and Obama: the Trust Evapor...
6874    Mike McPhate California Today: On the Ground i...
Name: contents, Length: 6875, dtype: object


In [ ]:
# separating the data and label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [ ]:
print(X)
print(Y)

        id                                              title  \
0        0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1        1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2        2                  Why the Truth Might Get You Fired   
3        3  15 Civilians Killed In Single US Airstrike Hav...   
4        4  Iranian woman jailed for fictional unpublished...   
...    ...                                                ...   
6870  6870  Andrew Garfield, Spiritual Ghost in the Hollyw...   
6871  6871  Harry Reid Vindicated: The FBI Is Sitting On I...   
6872  6872  Over 12,000 Tweets Call For Trump’s Assassinat...   
6873  6873              Putin and Obama: the Trust Evaporates   
6874  6874  California Today: On the Ground in Oroville - ...   

                  author                                               text  \
0          Darrell Lucus  House Dem Aide: We Didn’t Even See Comey’s Let...   
1        Daniel J. Flynn  Ever get the feeling your life circ

STEMMING
It is the process of reducing a word to its root words.

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_dataset['contents'] = news_dataset['contents'].apply(stemming)

In [ ]:
print(news_dataset['contents'])

0       darrel lucu hous dem aid even see comey letter...
1       daniel j flynn flynn hillari clinton big woman...
2                  consortiumnew com truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
6870    cara buckley andrew garfield spiritu ghost hol...
6871    jameson parker harri reid vindic fbi sit info ...
6872    jack hadfield tweet call trump assassin breitbart
6873                ray mcgovern putin obama trust evapor
6874    mike mcphate california today ground orovil ne...
Name: contents, Length: 6875, dtype: object


In [ ]:
news_dataset.loc[:, 'label'] = pd.to_numeric(news_dataset['label'], errors='coerce').fillna(0).astype(int)

In [ ]:
# separating the data and label
X = news_dataset['contents'].values;
Y = news_dataset['label'].values;

In [ ]:
# displaying the content data
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'jack hadfield tweet call trump assassin breitbart'
 'ray mcgovern putin obama trust evapor'
 'mike mcphate california today ground orovil new york time']


In [ ]:
# displaying the label data
print(Y)

[1 0 1 ... 0 1 0]


In [ ]:
Y.shape


(6875,)

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

In [ ]:
print(X)

  (0, 9577)	0.28300493715220104
  (0, 1550)	0.3956317020896285
  (0, 4723)	0.25143648120340895
  (0, 5289)	0.2982110554312007
  (0, 1844)	0.24773566132517855
  (0, 8211)	0.2572148146148013
  (0, 3063)	0.23546444521988447
  (0, 186)	0.2743212698775191
  (0, 2358)	0.26709230161809977
  (0, 4312)	0.22029791817933728
  (0, 5460)	0.341420613781976
  (0, 2239)	0.341420613781976
  (1, 1190)	0.15202385374005237
  (1, 1395)	0.37943881270227026
  (1, 10202)	0.28952333639839944
  (1, 946)	0.2852011264648358
  (1, 1749)	0.18937471191077962
  (1, 4205)	0.18889592190940427
  (1, 3410)	0.7223448465320816
  (1, 2219)	0.2728093882480799
  (2, 3338)	0.3836416516236548
  (2, 3692)	0.3405998803785845
  (2, 5888)	0.49676783555995124
  (2, 9532)	0.420672763561579
  (2, 1832)	0.32005063996301913
  :	:
  (6871, 7960)	0.217974150331673
  (6871, 2629)	0.1693633053604963
  (6871, 9523)	0.11311799791822298
  (6872, 3973)	0.5186338380883961
  (6872, 540)	0.4595565835389122
  (6872, 1372)	0.3305124501269484
  (6872

Splitting the dataset to training and testing data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=2)

Training the Model using Logistic Regression method

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluating the Model

In [ ]:
# accuracy score of the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.976909090909091


In [ ]:
# accuracy score of the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the testing data : ', test_data_accuracy)

Accuracy score of the testing data :  0.9549090909090909


Making a Predictive System

In [ ]:
X_new = X_test[2]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [ ]:
print(Y_test[2])

0
